# ScanSet workflow and FEDRA emrec applications

Here, the standard reconstruction workflow is illustrated. You may notice they are all bash commands, since we usually perform these operation from the terminal, to build our filesystem

## Creating working folder, getting data

We will use data from SNDCosmics commissionining (first 5 films of WALL1). We will make reconstruction only in 1 cm2 surface, for time and memory reasons.

We create a folder named SNDCosmics_tutorial/b000001 in our filesystem, then we create a link to the data

Plate numbers are named as p001, p002...p005

In [ ]:
!mkdir $CERNBOX_HOME/SNDCosmics_tutorial
!mkdir $CERNBOX_HOME/SNDCosmics_tutorial/b000001
!mkdir $CERNBOX_HOME/SNDCosmics_tutorial/b000001/p00{1..5}

we then create a soft link to our raw data in the working folder.

Raw data needs to have name format as 

1.1.0.0.raw.root

The four digits convention is:

* brick number (1, as in b000001);
* plate number (1, as in p001);
* major index (optional);
* minor index (optional);

Major and minor indices are useful to have multiple instances of the set. For now, just set them both to 0

In [ ]:
%%bash
for i in $(seq -w 1 5);
do
 ln -s /eos/experiment/sndlhc/emulsionData/SNDCosmics/b000001/p00$i/1.$i.0.0.raw.root $CERNBOX_HOME/SNDCosmics_tutorial/b000001/p00$i/1.$i.0.0.raw.root
done

In [ ]:
!ls $CERNBOX_HOME/SNDCosmics_tutorial/b000001

## Fitting base-tracks couples (linking)

Now we connect the information from the two layers of each film. We copy the example of parameters into the working folder

In [ ]:
!cp *.rootrc $CERNBOX_HOME/SNDCosmics_tutorial/b000001

Let us have a look at this file

In [ ]:
import os
os.chdir(os.environ["CERNBOX_HOME"]+"/SNDCosmics_tutorial/b000001")

In [ ]:
!ls
!cat link.rootrc

We can see that only a small area is selected (Header CUT). Other important parameters are:

* ICUT, selecting the segments
* Shr0 and DShr, the initial point of shrinkage and the shrinkage tolerance
* ThetaLimits, setting the angular limits


Let us make a set and proceed to the linking

In [ ]:
!makescanset -set=1.0.0.0 -dzbase=175 -from_plate=5 -to_plate=1 -v=2

In [ ]:
!emlink -set=1.0.0.0 -new -v=0

In [ ]:
import ROOT as r
import fedrarootlogon


In [ ]:
couplesfile = r.TFile.Open("p001/1.1.0.0.cp.root")
reportcanvas = couplesfile.Get("report")
reportcanvas.Draw()

In [ ]:
!ls

## Plate to plate alignment
Finding affine transformation between plates. Parameters are stored in rootrc file

We will perform here a 3 steps alignment, with progressive more restrictive pair of plates

Selection of couples is performed in the **readCPcut** string

### First alignment

Very small angle, less than 100 mrad. 

DZ correction is not computed here. 

Used to find rough xy offset. 

Notice: here, the -reset option is applied. This is done only in the first stage, as it will **reset all previous affine transformations** (we start from scratch).

In [ ]:
!cat first_align.rootrc

In [ ]:
%%bash
cp first_align.rootrc align.rootrc
makescanset -set=1.0.0.0 -dzbase=175 -dz=-1350 -from_plate=5 -to_plate=1 -v=2 -reset

In [ ]:
!emalign -set=1.0.0.0 -new -v=0

In [ ]:
!cp b000001.0.0.0.align.pdf firstalign_5_1.pdf

### Second alignment

Here, we do a larger angle interval. 

We also usually align only in a small area (1 cm2). In this case, we already have only a small area, so no position cut is applied


In [ ]:
!cat second_align.rootrc

In [ ]:
%%bash
cp second_align.rootrc align.rootrc
makescanset -set=1.0.0.0 -dzbase=175 -dz=-1350 -from_plate=5 -to_plate=1 -v=2

In [ ]:
%%bash
emalign -set=1.0.0.0 -new -v=0
cp b000001.0.0.0.align.pdf secondalign_5_1.pdf

### Third alignment

Last alignment, and the most accurate (the sigma and the maximum offset are set lower).

It is usually performed in the whole surface


In [ ]:
!cat third_align.rootrc

In [ ]:
%%bash
cp third_align.rootrc align.rootrc
makescanset -set=1.0.0.0 -dzbase=175 -dz=-1350 -from_plate=5 -to_plate=1 -v=2

In [ ]:
%%bash
emalign -set=1.0.0.0 -new -v=0
cp b000001.0.0.0.align.pdf thirdalign_5_1.pdf

### Checking alignment reports
They can be found in produced pdfs, or in the al.root files in the AFF folder (**only the latest alignment!**)

In particular, check that

* there are clear peaks in the dy vs dx and dty vs dtx residuals after final alignment;
* the found dz value is "close"(i.e., not larger or lower than 100 micron) to the expected one (-1300 micron, for 1mm W plates)

In [ ]:
alreportfile = r.TFile.Open("AFF/1.3.0.0_1.2.0.0.al.root")
alreport = alreportfile.Get("report_al")
alreport.Draw()

## Track reconstruction
Finally, we can perform track reconstruction

In [ ]:
!makescanset -set=1.0.0.0 -from_plate=5 -to_plate=1 -v=2

In [ ]:
!emtra -set=1.0.0.0 -new -v=0

In [ ]:
trackfile = r.TFile.Open("b000001.0.0.0.trk.root")
tracktree = trackfile.Get("tracks")
tracktree.GetEntries()

You may notice that we did not provide a track.rootrc here.

If a parameter file is not provided, it will use the default ones, from the last version of the FEDRA library.

However, it is possible to check which parameters have been used, since they are stored in a .save.rootrc file (in this case, **track.save.rootrc**).

Let us just check the parameter file used

In [ ]:
!cat track.save.rootrc